In [ ]:
#IMPORT MODULES
from Bio import SeqIO
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gzip
import sys
import os
from collections import defaultdict
from Bio.Blast import NCBIXML
from Bio.SeqFeature import SeqFeature
import subprocess
import gzip
import pypangraph as pp
from collections import Counter
import csv
import re
import itertools
import seaborn as sns
import pickle
import os

#IMPORT FILES
meta_data='/Users/jferrare/Documents/Good Lab Work/Gut Phage/uhgv_metadata.tsv.gz' #874051 rows
genomes='/Users/jferrare/Documents/Good Lab Work/Gut Phage/uhgv_mgv.fna.gz'
ani_reults='/Users/jferrare/Documents/Good Lab Work/Gut Phage/combined_uhgv_mgv_ani.txt' #30700847 rows
blast_results='/Users/jferrare/Documents/Good Lab Work/Gut Phage/combined_uhgv_mgv_blast_results.txt' #165831139 rows
ani_columns=['query_id', 'subject_id', 'num_alns', 'percent_identity', 'query_coverage', 'target_coverage']
blast_columns=['query_id','subject_id','percent_identity','length','mismatch','gapopen','qstart','qend','sstart','send','evalue','bitscore'] 
meta_columns=['uhgv_genome','uhgv_votu,votu_representative','original_study_alias','original_id','genome_length','is_jumbo_phage','gc_percent','checkv_quality','checkv_completeness','checkv_completeness_method','checkv_trimmed','viral_confidence','genomad_virus_score','genomad_virus_hallmarks','genomad_plasmid_hallmarks','viralverify_prediction','viralverify_score','checkv_viral_markers','checkv_host_markers','cds_count','cds_density','avg_cds_length','genetic_code',	'is_recoded','trna_count_total','trna_count_suppressor','sra_run','sra_sample','biosample','country	latitude','longitude']


In [ ]:
#for each strain create map of core_genome position to original alignment position
#we do this by original genome-->block loc/strand-->block_position_in_core_alignment
#THIS VERSION USES PANGRAPH CORE GENOME ALIGNMENT FUNCTION

input_path='/Users/jferrare/Documents/Good Lab Work/Gut Phage/January_2025/large_vOTU_pangraphs/pangraph_vOTU-000006_n249.json'
output_file='test.json'
# Load the Pangraph
pan = pp.Pangraph.load_json(input_path)
loc = pp.Locator(pan)

# Extract paths and block statistics
pd = pan.to_paths_dict()
bs = pan.to_blockstats_df()

# Filter for core blocks
bs_core = bs[bs['core'] == True]
core_blocks = bs_core.index

# Get all strain names
strains = list(pd.keys())

reference_strain = strains[0]

#Create dictionary from core genome block and block position to core alignment position
core_block_position_to_core_alignment = {}

# Use the reference strain's path to define core genome block order
reference_path = pd[reference_strain]
core_reference_path = [x for x in reference_path if x in core_blocks]

offset = 0  # running coordinate in the core alignment

for block_id in core_reference_path:
    bl = pan.blocks[block_id]
    aln = bl.generate_alignment()
    block_len = len(aln[0])

    for pos_in_block in range(0, block_len):  
        key=offset + pos_in_block+1
        value = (block_id, pos_in_block+1)
        core_block_position_to_core_alignment[key] = value

    offset += block_len
core_alignment_length = offset


output_dir = '/Users/jferrare/Downloads/'
for strain1 in strains:
    original_alignment_to_block={}
    for i in range(0, 1000000000):
        try:
            bl_id, bl_pos, occ = loc.find_position(strain=strain1, pos=i)
            original_alignment_to_block[(bl_id, bl_pos)] = i
        except:
            break

    core_alignment_to_original_alignment = {}
    count=0
    for i in range(1, core_alignment_length+1):
        try:
            value=core_block_position_to_core_alignment[i]
            mapped_position = original_alignment_to_block[value]
            core_alignment_to_original_alignment[i]=mapped_position
            count+=1
        except KeyError:
            continue
    #create txt file and save dictionary
    output_file = os.path.join(output_dir, f'{strain1}_core_alignment_to_original.pkl')
    with open(output_file, 'wb') as f:
        pickle.dump(core_alignment_to_original_alignment, f)


